In [5]:
import ssl
import re
import nltk
import warnings
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')
ssl._create_default_https_context = ssl._create_unverified_context

# Merging Datasets Together

There are currently four different datasets we are using (along with some of their columns):

| Oct 2017  | Nov 2017 - Dec 2017 | Sept 2018 - Feb 2019 | Oct 2019 |
| --- | --- | --- | --- |
| 350K | 390K | 695K | 15K |
| id, date of tweet, text | text, favorited, created | text, location, created | text, language, created at |

We want the date of the tweet and the text content of the tweet itself. We want to focus on tweets are in English.

In [9]:
# Import the datasets
df_oct17 = pd.read_csv('raw_data/oct2017.csv')
df_novdec17 = pd.read_csv('raw_data/novdec17.csv')
df_sept18feb19 = pd.read_csv('raw_data/sept2018feb2019.csv')
df_oct19 = pd.read_csv('raw_data/oct2019.csv')

# Drop columns from each dataframe as necessary
df_oct17 = df_oct17.drop(columns = ['id', 'insertdate', 'twitterhandle', 'followers', 'hashtagsearched',
                                   'tweetid', 'lastcontactdate', 'lasttimelinepull', 'lasttimetweetsanalyzed',
                                   'numberoftweetsanalysed', 'numberoftweetsabouthash', 'actualtwitterdate'])
df_oct17 = df_oct17.loc[:, ~df_oct17.columns.str.contains('^Unnamed')]
df_novdec17 = df_novdec17.drop(columns = ['favorited', 'favoriteCount', 'replyToSN', 'truncated', 'replyToSID',
                                         'id', 'replyToUID', 'statusSource', 'screenName', 'retweetCount',
                                         'isRetweet', 'retweeted', 'longitude', 'latitude'])
df_novdec17 = df_novdec17.loc[:, ~df_novdec17.columns.str.contains('^Unnamed')]
df_sept18feb19 = df_sept18feb19.drop(columns = ['status_id', 'favorite_count', 'retweet_count', 'location',
                                                'followers_count', 'friends_count', 'statuses_count', 'category'])
df_oct19 = df_oct19.drop(columns = ['Id', 'Lenght', 'Source', 'Favorite_count', 'Retweet_count'])
df_oct19 = df_oct19[df_oct19['Lang'] == 'en']
df_oct19 = df_oct19.drop(columns = ['Lang'])

                           dateoftweet  \
0       Thu Feb 08 22:36:52 +0000 2018   
1       Thu Feb 08 22:36:52 +0000 2018   
2       Thu Feb 08 22:37:00 +0000 2018   
3       Thu Feb 08 22:37:02 +0000 2018   
4       Thu Feb 08 22:37:11 +0000 2018   
...                                ...   
410641  Mon Oct 30 20:05:32 +0000 2017   
410642  Mon Oct 30 20:05:37 +0000 2017   
410643  Mon Oct 30 20:05:41 +0000 2017   
410644  Mon Oct 30 20:05:44 +0000 2017   
410645  Mon Oct 30 20:05:48 +0000 2017   

                                                     text  
0       RT @IxAmandaDelgado: @Navegaciones @FelipeCald...  
1       RT @alexwitze: .@NSF will require institutions...  
2       Listening to the awesome feminist scholar Cynt...  
3       ??????????????????????????????????????????????...  
4       RT @AlbertoBernalLe: ?A ver, donde est?n todas...  
...                                                   ...  
410641  RT @bre_stewart30: There is no easy way to tel...  
410642  @floral

In [16]:
print()
print(df_oct19)

                                                    Text        Created_at  \
0       @Rightsatbirth: "What he did was manipulative...  10/16/2019 22:33   
1      Happy 94th Birthday\0\x9f\x8e\x82 to @_AngelaL...  10/16/2019 22:33   
2       @RepLoriTrahan: Two years ago #MeToo woke up ...  10/16/2019 22:33   
3       @RituG15: #TimesUp #MeTooVoter  #MenToo #Meto...  10/16/2019 22:33   
4       @ambertamblyn: Months before MeToo broke us w...  10/16/2019 22:33   
...                                                  ...               ...   
15049  advocate4victim: #DYK: October 15 is the secon...  10/15/2019 17:46   
15050  TaranaBurke: Tonight, on the anniversary of #m...  10/15/2019 17:46   
15051  Meet Alki David: The self-appointed ambassador...  10/15/2019 17:46   
15052  IreneWoodbury: 2\x80\x9cGreat Read2\x806will l...  10/15/2019 17:46   
15053  manny_ottawa: Canadians don2\x80\x99t TRUST Tr...  10/15/2019 17:46   

      Lang  
0       en  
1       en  
2       en  
3       en 

                                                    Text        Created_at
0       @Rightsatbirth: "What he did was manipulative...  10/16/2019 22:33
1      Happy 94th Birthday\0\x9f\x8e\x82 to @_AngelaL...  10/16/2019 22:33
2       @RepLoriTrahan: Two years ago #MeToo woke up ...  10/16/2019 22:33
3       @RituG15: #TimesUp #MeTooVoter  #MenToo #Meto...  10/16/2019 22:33
4       @ambertamblyn: Months before MeToo broke us w...  10/16/2019 22:33
...                                                  ...               ...
15049  advocate4victim: #DYK: October 15 is the secon...  10/15/2019 17:46
15050  TaranaBurke: Tonight, on the anniversary of #m...  10/15/2019 17:46
15051  Meet Alki David: The self-appointed ambassador...  10/15/2019 17:46
15052  IreneWoodbury: 2\x80\x9cGreat Read2\x806will l...  10/15/2019 17:46
15053  manny_ottawa: Canadians don2\x80\x99t TRUST Tr...  10/15/2019 17:46

[15054 rows x 2 columns]
